In [1]:
from pyscf import gto, dft, scf

In [2]:
from PySCFAdapter import PySCFAdapter
from DQCAdapter import DQCAdapter

In [3]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

h2_system = "H 0 0 0; H 1.4 0 0"

n2_system = "N 0 0 0; N 2.07 0 0"

ammonia = """
N  0.000  0.000  0.000; 
H  0.000 -1.772 -0.721; 
H  1.535  0.886 -0.721; 
H -1.535  0.886 -0.721
"""

In [4]:
from math import pi

factor = -3.0/4.0 * (3.0 / pi)**(1.0/3.0)
power = 4.0/3.0

In [5]:
import torch
import dqc
import dqc.xc
import dqc.utils


class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a_par, p_par):
        super().__init__()
        self.a_par = a_par
        self.p_par = p_par
        self.number_of_parameters = 2

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a_par * rho ** self.p_par
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p_par
            elif number_of_parameter == 1: # parameter p
                return self.a_par * torch.log(rho) * rho ** self.p_par
            
a_par = torch.nn.Parameter(torch.tensor(factor, dtype=torch.double))
p_par = torch.nn.Parameter(torch.tensor(power, dtype=torch.double))
myxc = MyLDAX(a_par, p_par)

In [6]:
mol_dqc = dqc.Mol(moldesc=water, basis="6-31G")
qc_dqc = dqc.KS(mol_dqc, xc=myxc).run()
ene = qc_dqc.energy()
dqc_adapter = DQCAdapter(qc_dqc)

density info torch.Size([52366])
started the four term integral computation for torch.Size([52366, 13])...
g_p: tensor([ 10.9490, -11.0801], dtype=torch.float64)
g_x^t norm: 139.44220156340134
lambda norm: 80.89165246224454
f_p norm: 44.59154644003647
lambda_t * f_p: tensor([-2.6071e-15,  9.0339e-15], dtype=torch.float64)
f_x^-t norm: 1007.5233081497582
f_x^-1 * f_p norm: 18.110984676464824
g_x^t * f_x^-1 * f_p: tensor([-4.1455e-16, -3.3245e-15], dtype=torch.float64)


In [7]:
# print("Modified overlap:\n", dqc_adapter.get_overlap_matrix())
# print("Modified Fockian:\n", dqc_adapter.get_fockian())
# print("Orbital energies", dqc_adapter.get_orbital_energies())
# print("Modified orbital coefficients:\n", dqc_adapter.get_orbital_coefficients())
# print("Orbital occupancies", dqc_adapter.get_orbital_occupancy())
# print("Number of occupied orbitals:", dqc_adapter.get_number_of_occupied_orbitals())
# print("Number of all orbitals:", dqc_adapter.get_number_of_all_orbitals())
# print("Number of DFT parameters:", dqc_adapter.get_number_of_parameters())
# print("Derivative of exc with respect to parameters:\n", dqc_adapter.get_derivative_of_exc_wrt_theta())
# print("Derivative of vxc with respect to parameters:\n", dqc_adapter.get_derivative_of_vxc_wrt_theta())

In [8]:
import numpy

def custom_eval_xc(a, p):
    def eval_xc(xc_code, rho, *args, **kwargs):
        exc = a * rho ** (p - 1.0)
        vrho = a * p * rho ** (p - 1.0)
        vxc = (vrho, None, None, None)
        fxc = a * p * (p - 1.0) * rho ** (p - 2.0)  # 2nd order functional derivative
        return exc, vxc, fxc, None
    def eval_xc_wrt_a(xc_code, rho, *args, **kwargs):
        dexc_wrt_da = rho ** (p - 1.0)
        dvrho_wrt_da = p * rho ** (p - 1.0)
        dvxc_wrt_da = (dvrho_wrt_da, None, None, None)
        return dexc_wrt_da, dvxc_wrt_da, None, None
    def eval_xc_wrt_p(xc_code, rho, *args, **kwargs):
        dexc_wrt_dp = a * numpy.log(rho) * rho ** (p - 1.0)
        dvrho_wrt_dp = a * p * numpy.log(rho) * rho ** (p - 1.0)
        dvxc_wrt_dp = (dvrho_wrt_dp, None, None, None)
        return dexc_wrt_dp, dvxc_wrt_dp, None, None
    eval_xc.number_of_parameters = 2
    eval_xc.wrt = [eval_xc_wrt_a, eval_xc_wrt_p]
    return eval_xc

In [9]:
mol = gto.M(atom = water, basis="6-31G", unit = 'Bohr')
calc = dft.RKS(mol)
calc = calc.define_xc_(custom_eval_xc(factor, power), 'LDA')
ene = calc.kernel()

converged SCF energy = -75.154705340357


In [10]:
adapter = PySCFAdapter(calc)

Shape of dE/depsilon tensor: (5,)
Shape of elrep tensor: (13, 5, 13, 5)
Shape of dvxc/drho tensor: (13, 5, 13, 5)
Shape of fourDtensor: (13, 5, 13, 5)
Shape of dE/dC: (5, 13)
Shape of expand_dim'ed dE/depsilon: (5, 1)
Shape of dRoothan/dC: (13, 5, 13, 5)
Shape of dRoothan/depsilon: (13, 5, 5)
g_p: [ 10.94899516 -11.08012795]
g_x^t norm: 107.94364724360116
lambda norm: 80.8915693478313
f_p norm: 31.60508759656454
lambda_t * f_p: [ 3.06893295e-06 -5.55802934e-07]
f_x^-t norm: 3429.066014605868
f_x^-1 * f_p norm: 565.2055150615222
g_x^t * f_x^-1 * f_p: [ 3.06893295e-06 -5.55802938e-07]


In [11]:
elrep = mol.intor('int2e', aosym='s1')
for i in range(adapter.get_number_of_all_orbitals()):
    for j in range(adapter.get_number_of_all_orbitals()):
        for k in range(adapter.get_number_of_all_orbitals()):
            for l in range(adapter.get_number_of_all_orbitals()):
                assert(abs(elrep[i,j,k,l] - elrep[j,i,l,k])<10**-9),(elrep[i,j,k,l],elrep[j,i,l,k])

In [12]:
# print("Initial overlap:\n", calc.get_ovlp())
# print("Modified overlap:\n", adapter.get_overlap_matrix())
# print("Initial Fockian:\n", calc.get_fock())
# print("Modified Fockian:\n", adapter.get_fockian())
# print("Orbital energies", adapter.get_orbital_energies())
# print("Initial orbital coefficients:\n", scf.hf.kernel(calc)[3])
# print("Modified orbital coefficients:\n", adapter.get_orbital_coefficients())
# print("Orbital occupancies", adapter.get_orbital_occupancy())
# print("Number of occupied orbitals:", adapter.get_number_of_occupied_orbitals())
# print("Number of all orbitals:", adapter.get_number_of_all_orbitals())
# print("Number of DFT parameters:", adapter.get_number_of_parameters())
# print("Derivative of exc with respect to parameters:\n", adapter.get_derivative_of_exc_wrt_theta())
# print("Derivative of vxc with respect to parameters:\n", adapter.get_derivative_of_vxc_wrt_theta())

$FC=SCE$

$FS^{-\frac{1}{2}}S^{\frac{1}{2}}C=S^{\frac{1}{2}}S^{\frac{1}{2}}CE$

$\hat{C} = S^{\frac{1}{2}}C$

$FS^{-\frac{1}{2}}\hat{C}=S^{\frac{1}{2}}\hat{C}E$

$S^{-\frac{1}{2}}FS^{-\frac{1}{2}}\hat{C}=\hat{C}E$

$S^{-\frac{1}{2}}FS^{-\frac{1}{2}} = \hat{F}$

$\hat{F}\hat{C}=\hat{C}E$

In [13]:
print(numpy.linalg.eigh(dqc_adapter.get_four_center_xc_tensor().numpy().reshape((13*5,13*5)))[0])
print(numpy.linalg.eigh(adapter.get_four_center_xc_tensor().reshape((13*5,13*5)))[0])
numpy.abs(numpy.linalg.eigh(dqc_adapter.get_four_center_xc_tensor().numpy().reshape((13*5,13*5)))[0] - numpy.linalg.eigh(adapter.get_four_center_xc_tensor().reshape((13*5,13*5)))[0]).sum()

[-4.07394430e-01 -3.67833515e-01 -9.89961729e-02 -6.38375115e-02
 -5.86494465e-02 -4.29681534e-02 -3.25763093e-02 -2.70500610e-02
 -2.41909732e-02 -2.04925080e-02 -1.76165211e-02 -1.33652385e-02
 -1.18107922e-02 -9.43306814e-03 -8.66765025e-03 -8.32608740e-03
 -6.14288220e-03 -5.24462515e-03 -4.59362050e-03 -4.16485778e-03
 -3.07644752e-03 -2.72385482e-03 -2.64043467e-03 -2.29814745e-03
 -1.99110246e-03 -1.71809267e-03 -1.58614207e-03 -1.38817027e-03
 -1.14515469e-03 -9.54436433e-04 -7.43045078e-04 -6.86294764e-04
 -6.31744877e-04 -5.94207239e-04 -4.71212013e-04 -4.23126874e-04
 -3.32834739e-04 -3.10476143e-04 -2.88206522e-04 -2.23901406e-04
 -1.59284150e-04 -1.22579359e-04 -9.18320006e-05 -8.54198315e-05
 -5.92528834e-05 -5.54448458e-05 -4.75689379e-05 -3.44254325e-05
 -2.43602178e-05 -2.03723452e-05 -1.55348416e-05 -1.17212904e-05
 -1.00829601e-05 -6.22115153e-06 -1.62211100e-06 -5.98388647e-16
 -8.59327007e-17 -6.55255769e-17 -9.28153348e-18  1.04680942e-18
  1.87298826e-17  4.47416

0.7604358848137946